# Optimization of OT2 transfer paramters of viscous liquids guided by Baysesian Optimization

## Summary

This notebook objective is to generate new suggestions of aspiration and dispense rates that will minimize the tansfer error while minimizing the time of transfer of a viscous liquid. 
The code is strucutred as follows:
1.  Fisrt section is for importing the relevant packages to perform BO, inclduing Pytorch and BOtorch
2.  Second section includes the definition of the BO_LiqTransfer class that includes the method optimized_suggestions() that generates BO optimized aspiration and dispense rate values for a particular data set.
3.  Third section includes the code to run in conjuction with the OT2 BO optimization of a viscous liquid. The steps for the optimziation are:

    i. Initilize a BO_LiTransfer objecet and load initilization data using data_from_csv() method

    ii. Run optimized_suggestions() method

    iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook

    iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.
    
    v. Iterate through steps ii-iV
 

# 1. Imports

In [3]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time
from datetime import date

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

from gpytorch.constraints import GreaterThan
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


# 2. BO_LiqTransfer class

In [4]:
class BO_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error','time_asp_1000']
        self.bmax = 1.25
        self.bmin = 0.1
        self._latest_suggestion = None
        self._latest_volume = None
        self._latest_acq_value = None
        self.mean_volumes = [300,500,1000]
    
   
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']
        if 'acq_value' not in df.columns:
            df['acq_value'] = None

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df
        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                    df_duplicates.loc[index,'%error'] = -mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean
 
        




    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        data = data.loc[:,['liquid','pipette','volume','aspiration_rate','dispense_rate','blow_out_rate','delay_aspirate','delay_dispense','delay_blow_out','%error']]
        self.set_data(data)


    def update_data(self,error,volume= 1000):
        self._latest_volume = volume
        updated_data = pd.concat([self._data,self._data.iloc[[-1]]],ignore_index=True)
        updated_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        updated_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate']
        updated_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate']
        updated_data.loc[updated_data.last_valid_index(),'acq_value']  = self._latest_acq_value
        updated_data.loc[updated_data.last_valid_index(),'%error'] = error
        self.set_data(updated_data)
        return self._data
                                

    def xy_split(self):
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return torch.vstack([x_train[0]*self.bmin, x_train[0]*self.bmax])



    def fit_surrogate(self):
        x_train, y_train = self.xy_split()
        x_train = torch.tensor(x_train.to_numpy(dtype=float), **tkwargs)
        y_train = torch.tensor(y_train.to_numpy(dtype=float), **tkwargs)
        y_train[:,0] = -torch.absolute(y_train[:,0])
        y_train[:,1] = -torch.absolute(y_train[:,1])

        problem_bounds = self.set_bounds(x_train)
        time_upper = 1000/problem_bounds[0][0] +1000/problem_bounds[0][1] + 10
        error_upper = y_train[:,0].abs().min()*1.25
        ref_point = torch.tensor([-error_upper,-time_upper], **tkwargs)

        train_x_gp = normalize(x_train, problem_bounds)
        models = []
        for i in range(y_train.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, y_train[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model1 = ModelListGP(*models)
        mll1 = SumMarginalLogLikelihood(model1.likelihood, model1)

        fit_gpytorch_model(mll1)
    
        return model1, ref_point, train_x_gp, problem_bounds
    
    def optimized_suggestions(self, random_state= 42):
        if random_state != None:
            torch.manual_seed(random_state) 
        standard_bounds = torch.zeros(2, len(self.features), **tkwargs)
        standard_bounds[1] = 1
        model1, ref_point, train_x_gp, problem_bounds = self.fit_surrogate()
        acq_func1 = qNoisyExpectedHypervolumeImprovement(
        model=model1,
        ref_point=ref_point, # for computing HV, must flip for BoTorch
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(sample_shape=512),  # determines how candidates are randomly proposed before selection
        objective=IdentityMCMultiOutputObjective(outcomes=np.arange(len(self.objectives)).tolist()), # optimize first n_obj col 
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        sobol1 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol2 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol_all = torch.vstack([sobol1, sobol2])
            
        acq_value_list = []
        for i in range(0, sobol_all.shape[0]):
            with torch.no_grad():
                acq_value = acq_func1(sobol_all[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())
                
        # filter the best 12 QMC candidates first
        sorted_x = sobol_all.cpu().numpy()[np.argsort((acq_value_list))]
        qnehvi_x = torch.tensor(sorted_x[-12:], **tkwargs)  
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(qnehvi_x, bounds=problem_bounds)
        new_x = pd.DataFrame(new_x.numpy(),columns=['aspiration_rate','dispense_rate'])
        new_x['acq_values'] = sorted(acq_value_list, reverse=True)[:12]
        self._latest_suggestion = new_x[['aspiration_rate','dispense_rate']].iloc[0]
        self._latest_acq_value = new_x['acq_values'].iloc[0] 
        return new_x
        





        
    
    


# 3. Guided BO optimization
i.   Create BO_LiqTransfer object and load data set.

Please set liquid name and volume to transfer according to the experiment.

In [6]:
# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = BO_LiqTransfer(liquid_name)
liq.data_from_csv(r'C:\Users\amdm_\OneDrive\Documents\GitHub\viscosity_liquid_transfer_Pablo\Opentrons_experiments\BOTorch_optimization\VS_code_csv\Viscosity_std_1275_exp3_withacq_modified_bounds.csv')
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.657700,None
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.657700,None
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.657700,None
3,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.657700,None
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.526160,None
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.526160,None
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.526160,None
7,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.526160,None
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578,None
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578,None



ii.   Run BO_LiqTransfer.optimized_suggestions() method to obtain optimized aspiration and dispense rates.



In [30]:
liq.optimized_suggestions()

,aspiration_rate,dispense_rate,acq_values
0,15.998308,2.435148,32.077965
1,12.638498,2.758106,26.952416
2,2.451890,10.873625,16.913759
3,6.194447,2.611040,14.307633
4,2.419982,10.585014,13.165620
5,11.103476,2.529725,10.642798
6,9.278880,2.919787,8.084997
7,14.084860,2.392505,7.766193
8,13.985054,2.357217,6.444701
9,8.532868,2.868048,6.435466


iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook.

iv. Update volume and latest %error obtained from the transfer using suggested aspiration and dispense rates. Repeat for 1000, 500, 100 uL

In [33]:
volume= 300
liq.update_data(-9.607753,volume)



,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.6577,None
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.6577,None
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.6577,None
3,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.6577,None
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.52616,None
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.52616,None
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.52616,None
7,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.52616,None
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578,None
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578,None


In [35]:
#save after each standard-experiment iteration
#Please check ;liquid name before saving to prevent accidental overwriting of previous data

liq._data.to_csv('C:\\Users\\amdm_\\OneDrive\\Documents\\GitHub\\viscosity_liquid_transfer_Pablo\\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\'+liquid_name+'_'+'exp3_withacq_modified_bounds_final.csv', index = False)

v. Iterate trhough last two code cells.

In [14]:
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000,acq_value
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.6577,None
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.6577,None
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.6577,None
3,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",22.061005,22.061005,0,5,5,0,-27.408934,100.6577,None
4,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.52616,None
5,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.52616,None
6,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.52616,None
7,Viscosity_std_1275,p1000,"mean[300, 500, 1000]",27.576257,27.576257,0,5,5,0,-30.110399,82.52616,None
8,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578,None
9,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578,None
